# Feast test on sales data

* Взял рабочие данные по продажам автозапчастей, создал паркетник ```/data/sales.parquet```
* В ```example_repo.py``` создал 2 фиче-вью и 1 он-деманд-фиче-вью
* В разделе [Get features from store](#Get-features-from-store) пример использования

## imports

In [1]:
from feast import (
    FeatureView,
    Field,
    FeatureStore
)
from feast.on_demand_feature_view import on_demand_feature_view
from feast.types import Float32, Float64, String
from datetime import timedelta

In [2]:
import pandas as pd
import numpy as np
import os

## get data

In [4]:
import psycopg2
host = ## 
port = "6432"
dbname = "dwh"
username = "zhukov.a"
pwd = os.getenv("PWD_CLOUD_AUTOSTELS")

In [5]:
conn = psycopg2.connect(
    "host='{}' port={} dbname='{}' user={} password={}".format(host, port, dbname, username, pwd)
)

In [6]:
import pandas.io.sql as sqlio

In [10]:
df =  sqlio.read_sql("select * FROM dwh_cdm.sale_order_calc_det sale limit 10000", conn)

/var/folders/wb/s5b9f5wx3kj_c83kz1ynmx300000gn/T/ipykernel_52923/3680025023.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df =  sqlio.read_sql("select * FROM dwh_cdm.sale_order_calc_det sale limit 10000", conn)


In [14]:
df.to_parquet("./data/sales.parquet")

In [3]:
df = pd.read_parquet("./data/sales.parquet")

In [20]:
df.head()

,sale_order_item_id,customer_id,supplier_top_id,order_item_work_dt,order_item_create_dt,order_price_with_tax_amt,order_price_no_tax_amt,order_qty_cnt,order_cost_with_tax_amt,order_cost_no_tax_amt,...,supp_cost_with_tax_amt,supp_cost_no_tax_amt,supp_cost_no_tax_amt_dds,valid_from_dttm,valid_to_dttm,md5_hash,insert_process_id,insert_dttm,update_process_id,update_dttm
0,91322375,262481,-1,2013-04-16 13:54:07,NaT,206.69,175.161017,1,206.69,175.161017,...,177.14,150.118644,NaN,1900-01-01,9999-12-31,1b49f3faff0b24ee3ad6b805ff2b9f7b,3447,2024-05-17 01:35:32.339873,3447,2024-05-17 01:35:32.339873
1,95352883,270590,-1,2013-11-06 21:06:09,NaT,710.72,602.305085,1,710.72,602.305085,...,618.92,524.508475,NaN,1900-01-01,9999-12-31,b5d65d83e79727454afb3fdf4fab393d,3447,2024-05-17 01:35:32.339873,3447,2024-05-17 01:35:32.339873
2,91286651,266948,-1,2010-12-20 15:03:10,NaT,205.99,174.567797,1,205.99,174.567797,...,175.74,148.932203,NaN,1900-01-01,9999-12-31,79358c12fa6ba30ceeecb2402d40021e,3447,2024-05-17 01:35:32.339873,3447,2024-05-17 01:35:32.339873
3,93466781,278626,-1,2018-02-13 09:16:41,2018-02-13 09:15:41,749.00,634.745763,1,749.00,634.745763,...,664.39,563.042373,NaN,1900-01-01,9999-12-31,4b9a0f9ad2598a0c23750a171a3cecfc,3447,2024-05-17 01:35:32.339873,3447,2024-05-17 01:35:32.339873
4,66970497,279624,1671,2021-05-15 13:35:07,2021-05-15 13:34:32,4160.00,3466.666667,1,4160.00,3466.666667,...,4037.37,3364.475000,3364.47,1900-01-01,9999-12-31,825b8cf6829230b6ed5c7eb3ce6c82b3,3447,2024-05-17 01:35:32.339873,3447,2024-05-17 01:35:32.339873


## Get features from store

In [13]:
store = FeatureStore(repo_path=".")

In [31]:
entity_df = pd.DataFrame.from_dict(
    {
        "customer_id": [262481],
        "event_timestamp": [
            datetime(2013, 4, 16, 13, 54, 7)
        ],
        "tax_value": 0.2,
    }
)

In [32]:
training_df = store.get_historical_features(
        entity_df=entity_df,
        features=[
            "sales_daily_stats:order_qty_cnt",
            "sales_daily_stats:order_cost_with_tax_amt",
            "no_tax_daily:order_cost_without_tax_amt",
        ],
    ).to_df()

In [33]:
training_df

,customer_id,event_timestamp,tax_value,order_qty_cnt,order_cost_with_tax_amt,order_cost_without_tax_amt
0,262481,2013-04-16 13:54:07+00:00,0.2,1,206.69,165.352


In [34]:
df[df.customer_id.isin([262481])].head(1)

,sale_order_item_id,customer_id,supplier_top_id,order_item_work_dt,order_item_create_dt,order_price_with_tax_amt,order_price_no_tax_amt,order_qty_cnt,order_cost_with_tax_amt,order_cost_no_tax_amt,...,supp_cost_with_tax_amt,supp_cost_no_tax_amt,supp_cost_no_tax_amt_dds,valid_from_dttm,valid_to_dttm,md5_hash,insert_process_id,insert_dttm,update_process_id,update_dttm
0,91322375,262481,-1,2013-04-16 13:54:07,NaT,206.69,175.161017,1,206.69,175.161017,...,177.14,150.118644,NaN,1900-01-01,9999-12-31,1b49f3faff0b24ee3ad6b805ff2b9f7b,3447,2024-05-17 01:35:32.339873,3447,2024-05-17 01:35:32.339873
